## 数据处理

In [12]:
import pandas as pd
import io
import re
from datetime import datetime, timedelta
from typing import List, Union, Optional
from datetime import datetime

import os, re, json, time
import typing as T
import requests
from pathlib import Path
from openpyxl import Workbook, load_workbook

from openpyxl.utils import get_column_letter
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from data_processing import load_and_process,build_jsonl_for_range, save_jsonl


In [13]:
## 研发字典
speaker_map = {
    "16186514":   "研发peter本尊",
    "1655611808": "运营绾绾",
    "2073820674": "沙利文老师",
}
## 客服字典
MAPPING_FILE = "mapping地球1.xlsx"

##QQ的txt文件
pathtxt      = "1114《欢迎来到地球》测试1群.txt"

# 设定时间范围
start_time = "2025-10-23 00:00:00"
end_time   = "2025-10-24 00:00:00"

# 1) 拿到 JSONL（列表）
jsonl_lines01 = build_jsonl_for_range(
    pathtxt=pathtxt,
    mapping_file=MAPPING_FILE,
    speaker_map=speaker_map,
    start_time=start_time,
    end_time=end_time,
    return_str=False,   # 返回 list[str]
)

print(len(jsonl_lines01), "lines")
print(jsonl_lines01[0])



2779 lines
{"发言日期": "2025-10-23", "发言时间": "00:00:00", "玩家ID": "UKING(3155443546)", "玩家消息": "所以能选二级就二级是吗？"}


## 大模型分类

## 定义

In [14]:
"""
批处理 10000 条聊天数据（每批 100 条）：
- 模型#1：过滤非游戏相关（只保留相关 JSON 行，原样输出）
- 模型#2：提取高讨论的发言并分析
- 结果按word格式文档输出
"""
from model_classifyV1_Copy1_Copy1 import (
    load_system_prompt,
    build_user_prompt_filter,build_user_prompt_discuss,build_user_prompt_emotion,
    build_ingame_special,build_outgame_special,
    call_ark_chat_completions,  
    create_word_report_all, append_hot_grouped,append_emotion_grouped,insert_paragraph_after,
    append_ingame_special,append_outgame_special
)

## 设置参数

In [15]:
# ============= 你的模型与文件配置（改这里） =============
API_URL   = "https://ark.cn-beijing.volces.com/api/v3/chat/completions" 
API_KEY = "de91deb0-aae6-46cb-bac0-17ac3b6107f5" #API
V3_MODEL_ID= "ep-20251020160142-5d7hp"#接入点
V3_1_MODEL_ID = "ep-20251020160025-9p5tj"#接入点
R1_MODEL_ID = "ep-20251020160103-5n6g2"#接入点

PROMPT_MD_PATH01 = Path("提示词1.md")      # 模型#1 system 提示词（筛相关）
PROMPT_MD_PATH02 = Path("1热度讨论.md")      # 模型#2 system 提示词（热度事件分析）
PROMPT_MD_PATH03 = Path("2情绪波动.md")      # 模型#3 system 提示词（情绪高涨事件分析）
PROMPT_MD_PATH04 = Path("3公共事件_游戏相关.md")      # 模型#4 system 提示词（游戏内特殊事件分析）
PROMPT_MD_PATH05 = Path("4公共事件_非游戏.md")      # 模型#5 system 提示词（游戏外特殊事件分析）
WORD_FILE       = Path("事件分析10232.docx")   # 输出 Excel
BATCH_SIZE       = 250
SLEEP_BETWEEN    = 1   # 每批之间的间隔，防止QPS触发限流；按需调整
RETRIES          = 2
TEMPERATURE      = 0.20
MAX_TOKENS       = 16384
TIMEOUT_SEC      = 600
# =====================================================

## 开始运行

# 特殊事件

In [8]:
from tqdm import tqdm
import time

# ===== 加载提示词 =====
system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)               # 筛相关
system_prompt04 = load_system_prompt(PROMPT_MD_PATH04)      # 游戏相关特殊事件
system_prompt05 = load_system_prompt(PROMPT_MD_PATH05)      # 非游戏相关事件

total = len(jsonl_lines01)
total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
print(f"准备处理 {total} 条，共 {total_batches} 批。")

for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):

    # === 取本批 JSONLines（还是 list[str]） ===
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    batch_lines = jsonl_lines01[start:end]

    # 转成 JSONL 文本块
    batch_jsonl = "\n".join(batch_lines)

    # ==========================================================
    # 🟦 智能体 #1：筛相关 → output_filter（仍然是一个 JSONL 文本块）
    # ==========================================================
    user_prompt1 = build_user_prompt_filter(batch_lines)

    output_filter = call_ark_chat_completions(
        api_url=API_URL,
        api_key=API_KEY,
        model=V3_MODEL_ID,
        system_prompt=system_prompt01,
        user_prompt=user_prompt1,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        timeout=TIMEOUT_SEC,
    )

    if not output_filter:
        print(f"[批次 {b+1}] ❌ 筛相关无结果")
        continue

    # ==========================================================
    # 🟧 计算非相关事件输入（保持 JSON 格式）
    # ==========================================================

    # 将 output_filter 也按行切开，但保持 JSON 文本
    filtered_lines = output_filter.split("\n")
    # 删掉空行
    filtered_lines = [l for l in filtered_lines if l.strip()]
    # 用字符串差集找到“非游戏相关”文本
    # （注意：保持 JSON 格式，不破坏结构）
    non_related_lines = [l for l in batch_lines if l not in filtered_lines]
    non_related_jsonl = "\n".join(non_related_lines)

    # ==========================================================
    # 🟥 智能体 #2：游戏相关特殊事件（输入 = output_filter）
    # ==========================================================
    user_prompt4 = build_ingame_special(output_filter)

    output_ingame_special = call_ark_chat_completions(
        api_url=API_URL,
        api_key=API_KEY,
        model=V3_1_MODEL_ID,
        system_prompt=system_prompt04,
        user_prompt=user_prompt4,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        timeout=TIMEOUT_SEC,
    )

    print("\n==============================")
    print(f"🎮 批次 {b+1} —— 游戏相关特殊事件")
    print("==============================")
    print(output_ingame_special)

    # ==========================================================
    # 🟩 智能体 #3：非游戏相关特殊事件（输入 = non_related_jsonl）
    # ==========================================================
    if non_related_jsonl.strip():
        user_prompt5= build_outgame_special(non_related_jsonl)

        output_outgame_special = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt05,
            user_prompt=user_prompt5,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
        )

        print("\n==============================")
        print(f"🌍 批次 {b+1} —— 非游戏相关特殊事件")
        print("==============================")
        print(output_outgame_special)

    else:
        print(f"\n🌍 批次 {b+1} —— 没有非游戏相关内容")

    time.sleep(SLEEP_BETWEEN)


准备处理 376 条，共 2 批。


🔥 批处理进度:   0%|                                                                                                                                     | 0/2 [00:00<?, ?批/s]


🎮 批次 1 —— 游戏相关特殊事件
[
  {
    "事件名称": "被迫害妄想技能第一回合未生效BUG",
    "事件摘要": "多名玩家反馈大蝴蝶殖装的被迫害妄想技能描述为第一回合释放但实际未生效，客服确认并反馈问题",
    "玩家共识": "玩家普遍认为这是技能描述与实际效果不符的异常情况，需要修复",
    "事件影响": "玩家技能策略失效，需要重新调整战斗部署，客服介入收集信息进行修复",
    "讨论热度（量化）": "发言玩家总数：3 位，发言总量：5 条"
  },
  {
    "事件名称": "远古BUG导致游戏开局黑屏卡死",
    "事件摘要": "玩家反馈开局遭遇黑屏卡死问题，重启无效，客服建议重启但问题未解决",
    "玩家共识": "玩家认为这是已知的远古BUG，影响游戏正常进行，需要紧急修复",
    "事件影响": "玩家无法正常进行游戏，部分玩家选择暂停游玩等待更新修复",
    "讨论热度（量化）": "发言玩家总数：2 位，发言总量：4 条"
  }
]

🌍 批次 1 —— 非游戏相关特殊事件
[]


🔥 批处理进度:  50%|██████████████████████████████████████████████████████████████                                                              | 1/2 [05:15<05:15, 315.68s/批]


🎮 批次 2 —— 游戏相关特殊事件
[]

🌍 批次 2 —— 非游戏相关特殊事件
[]


🔥 批处理进度: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [08:02<00:00, 241.46s/批]


# 模型二 情绪高点事件

In [ ]:
from tqdm import tqdm
import time

# ... 前置：系统提示、create_intent_excel_styled(EXCEL_FILE)、jsonl_lines01 等

system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 热度事件分析
system_prompt03 = load_system_prompt(PROMPT_MD_PATH03)  # 情绪高点分析


total = len(jsonl_lines01)
if total == 0:
    print("没有可处理的数据。")
else:
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批（每批 {BATCH_SIZE} 条）。")


for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    batch_lines = jsonl_lines01[start:end]

    # --- 模型 #1：筛相关 ---
    user_prompt1 = build_user_prompt_filter(batch_lines)
    output_filter = call_ark_chat_completions(
        api_url=API_URL,
        api_key=API_KEY,
        model=V3_MODEL_ID,
        system_prompt=system_prompt01,
        user_prompt=user_prompt1,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        timeout=TIMEOUT_SEC,
        retries=RETRIES,
    )

    if not output_filter:
        tqdm.write(f"[批次 {b+1}] 模型#1 无有效输出，跳过。")
        continue

    # --- 模型 #3：情绪高点分析 ---
    user_prompt3 = build_user_prompt_emotion(output_filter)
    output_emotion = call_ark_chat_completions(
        api_url=API_URL,
        api_key=API_KEY,
        model=V3_MODEL_ID,
        system_prompt=system_prompt03,
        user_prompt=user_prompt3,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        timeout=TIMEOUT_SEC,
        retries=RETRIES,
    )

    # ⭐⭐⭐ 打印输出 ⭐⭐⭐
    print("\n=========================")
    print(f"🔥 批次 {b+1} 模型 #3（情绪高点分析）输出")
    print("=========================\n")
    print(output_emotion)

    # 防止 API 限流
    time.sleep(SLEEP_BETWEEN)

print("\n✅ 全部批次处理完成！")


准备处理 376 条，共 2 批（每批 250 条）。


🔥 批处理进度:   0%|                                                                                                                                     | 0/2 [00:00<?, ?批/s]

# 最终主循环

In [6]:
from tqdm import tqdm
import time

system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 热度事件分析
system_prompt03 = load_system_prompt(PROMPT_MD_PATH03)  # 情绪高点分析

create_word_report_grouped(WORD_FILE)  # 只创建一次

total = len(jsonl_lines01)
if total == 0:
    print("没有可处理的数据。")
else:
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批。")

for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    
    # === 取本批数据 ===
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    batch_lines = jsonl_lines01[start:end]

    # === 模型 #1：筛相关 ===
    output_filter = call_ark_chat_completions(
        api_url=API_URL,
        api_key=API_KEY,
        model=V3_MODEL_ID,
        system_prompt=system_prompt01,
        user_prompt=build_user_prompt_filter(batch_lines),
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        timeout=TIMEOUT_SEC,
        retries=RETRIES,
    )

    if not output_filter:
        tqdm.write(f"[批次 {b+1}] 模型 #1 无输出，跳过。")
        continue

    # === 模型 #2：热度分析 ===
    user_prompt2 = build_user_prompt_discuss(output_filter)
    output_discussion = call_ark_chat_completions(
        api_url=API_URL,
        api_key=API_KEY,
        model=V3_1_MODEL_ID,
        system_prompt=system_prompt02,
        user_prompt=user_prompt2,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        timeout=TIMEOUT_SEC,
        retries=RETRIES,
    )

    # === 模型 #3：情绪高点分析 ===
    user_prompt3 = build_user_prompt_emotion(output_filter)
    output_emotion = call_ark_chat_completions(
        api_url=API_URL,
        api_key=API_KEY,
        model=V3_MODEL_ID,
        system_prompt=system_prompt03,
        user_prompt=user_prompt3,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        timeout=TIMEOUT_SEC,
        retries=RETRIES,
    )

    # === 写入 Word（按你的格式） ===
    append_hot_grouped(WORD_FILE, output_discussion)
    append_emotion_grouped(WORD_FILE, output_emotion)

    time.sleep(SLEEP_BETWEEN)


print("\n🎉 全部批次处理完成！")
print(f"📄 Word 文件已生成：{WORD_FILE}")

准备处理 376 条，共 2 批。


🔥 批处理进度: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [09:15<00:00, 277.94s/批]


🎉 全部批次处理完成！
📄 Word 文件已生成：事件分析.docx


# 最终主循环（热度+情绪高点+游戏内外特殊事件）

In [17]:
from tqdm import tqdm
import time

system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 热度事件分析
system_prompt03 = load_system_prompt(PROMPT_MD_PATH03)  # 情绪高点分析
system_prompt04 = load_system_prompt(PROMPT_MD_PATH04)  # 游戏内特殊事件
system_prompt05 = load_system_prompt(PROMPT_MD_PATH05)  # 游戏外特殊事件

create_word_report_all(WORD_FILE)  # 只创建一次

total = len(jsonl_lines01)
if total == 0:
    print("没有可处理的数据。")
else:
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批。")

for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    
    # === 每批开始时重置这个变量，避免沿用上一次的值 
    output_outgame_special = None

    # === 取本批数据 ===
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    batch_lines = jsonl_lines01[start:end]

    # === 模型 #1：筛相关 ===
    output_filter = call_ark_chat_completions(
        api_url=API_URL,
        api_key=API_KEY,
        model=V3_MODEL_ID,
        system_prompt=system_prompt01,
        user_prompt=build_user_prompt_filter(batch_lines),
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        timeout=TIMEOUT_SEC,
        retries=RETRIES,
    )

    if not output_filter:
        tqdm.write(f"[批次 {b+1}] 模型 #1 无输出，跳过。")
        continue

    # === 模型 #2：热度分析 ===
    user_prompt2 = build_user_prompt_discuss(output_filter)
    output_discussion = call_ark_chat_completions(
        api_url=API_URL,
        api_key=API_KEY,
        model=V3_1_MODEL_ID,
        system_prompt=system_prompt02,
        user_prompt=user_prompt2,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        timeout=TIMEOUT_SEC,
        retries=RETRIES,
    )
   

    # === 模型 #3：情绪高点分析 ===
    user_prompt3 = build_user_prompt_emotion(output_filter)
    output_emotion = call_ark_chat_completions(
        api_url=API_URL,
        api_key=API_KEY,
        model=V3_MODEL_ID,
        system_prompt=system_prompt03,
        user_prompt=user_prompt3,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        timeout=TIMEOUT_SEC,
        retries=RETRIES,
    )
    
    # === 代码：非游戏相关发言提取 ===
    # 将 output_filter 也按行切开，但保持 JSON 文本
    filtered_lines = output_filter.split("\n")
    # 删掉空行
    filtered_lines = [l for l in filtered_lines if l.strip()]
    # 用字符串差集找到“非游戏相关”文本
    # （注意：保持 JSON 格式，不破坏结构）
    non_related_lines = [l for l in batch_lines if l not in filtered_lines]
    non_related_jsonl = "\n".join(non_related_lines)
    
    # === 模型 #4：游戏内特殊事件分析 ===
    user_prompt4 = build_ingame_special(output_filter)
    output_ingame_special = call_ark_chat_completions(
        api_url=API_URL,
        api_key=API_KEY,
        model=V3_MODEL_ID,
        system_prompt=system_prompt04,
        user_prompt=user_prompt4,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        timeout=TIMEOUT_SEC,
    )
    # === 模型 #5：游戏外特殊事件分析 ===
    if non_related_jsonl.strip():
        user_prompt5= build_outgame_special(non_related_jsonl)
        output_outgame_special = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt05,
            user_prompt=user_prompt5,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
        )
    # === 写入 Word（按你的格式） ===
    if output_discussion:
        append_hot_grouped(WORD_FILE, output_discussion)

    if output_emotion:
        append_emotion_grouped(WORD_FILE, output_emotion)

    if output_ingame_special:
        append_ingame_special(WORD_FILE, output_ingame_special)

    if output_outgame_special:
        append_outgame_special(WORD_FILE, output_outgame_special)

        
    time.sleep(SLEEP_BETWEEN)


print("\n🎉 全部批次处理完成！")
print(f"📄 Word 文件已生成：{WORD_FILE}")

准备处理 2779 条，共 12 批。


🔥 批处理进度: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [1:12:13<00:00, 361.15s/批]


🎉 全部批次处理完成！
📄 Word 文件已生成：事件分析10232.docx
